In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
from tensorflow import Session, Graph
from keras.layers.convolutional import Conv2D
from keras.layers import Input, Dense, Flatten, Lambda, add
from keras.optimizers import RMSprop, Adam
from keras.models import Sequential ,load_model, Model, model_from_json
from keras.backend.tensorflow_backend import set_session
from keras.models import model_from_json
from keras.utils import to_categorical
from keras.utils.training_utils import multi_gpu_model
from keras import backend as K

from skimage.color import rgb2gray
from collections import deque
from skimage.transform import resize
from PIL import Image as iimage
from models.cnn_model_LSTM_many_to_one import cnn_lstm
from matplotlib import style, gridspec                              

import matplotlib.pyplot as plt
import rospy
import scipy.misc
import numpy as np
import time
import random
import pickle
# import cv2
import copy

Using TensorFlow backend.


In [2]:
def getModel(img_width, img_height, img_channel, num_frames, output_dim,num_actions, model_path):
    model = cnn_lstm(img_width, img_height, img_channel, num_frames, output_dim, num_actions)
    
    if model_path:
        try:
            model.load_weights(model_path)
            print("Loaded model from {}".format(model_path))
        
        except:
            print("Impossible to find weight path. Train new model.")
    return model

In [ ]:
# load model trained with multi_gpus
time_to_collision = getModel(128, 128, 3, 6, 1,8, 'None')
time_to_collision_multi = multi_gpu_model(time_to_collision, gpus=2)
time_to_collision_multi.load_weights('./cnn_saved_models/GAN_model/ConvLSTM_V1.h5')

In [ ]:
# model for single
time_to_collision.save_weights('./cnn_saved_models/LSTM_models_real_data/ConvLSTM_V2_single.h5')

In [ ]:
class TestAgent:
    def __init__(self, action_size):
        self.state_size = (128, 160 , 6)
        self.action_size = action_size
        self.model = multi_gpu_model(self.build_model(), gpus = 2)
        " Erase the config and tf.initializer when you load another model by keras!!!"

    def build_model(self):
        input = Input(shape=self.state_size)
        h1 = Conv2D(32, (8, 8), strides = (8,8), activation = "relu", name = "conv1")(input)
        h2 = Conv2D(64, (4, 4), strides = (2,2), activation = "relu", name = "conv2")(h1)
        h3 = Conv2D(64, (3, 3), strides = (1,1), activation = "relu", name = "conv3")(h2)
        context = Flatten(name = "flatten")(h3)
        
        value_hidden = Dense(512, activation = 'relu', name = 'value_fc')(context)
        value = Dense(1, name = "value")(value_hidden)
        action_hidden = Dense(512, activation = 'relu', name = 'action_fc')(context)
        action = Dense(self.action_size, name = "action")(action_hidden)
        action_mean = Lambda(lambda x: tf.reduce_mean(x, axis = 1, keepdims = True), name = 'action_mean')(action) 
        output = Lambda(lambda x: x[0] + x[1] - x[2], name = 'output')([action, value, action_mean])
        model = Model(inputs = input, outputs = output)
        model.summary()
        
        return model 
      
    
    def get_action(self, history):
        flag = False
        if np.random.random() < 0.001:
            flag = True
            return random.randrange(8), flag 
        history = np.float32(history)
        q_value = self.model.predict(history)
        return np.argmax(q_value[0]), flag

    def load_model(self, filename):
        self.model.load_weights(filename)

In [ ]:
# Intial hovering for 2 seconds and collecting the data from the laser and camera
# Receivng the data from laser and camera
# checking the crash using crash_check function and if crash occurs, the simulation is reset
if __name__ == '__main__':
    # Check the gazebo connection    
    # Parameter setting for the simulation
    agent = TestAgent(action_size = 8)  ## class name should be different from the original one
    agent.load_model("/home/asl/machine_learning/project/Collision_Avoidance_depth_V5/save_model/D3QN_CP_V_1.h5")